In [7]:
from jupytercad import CadDocument
from functools import partial
import asyncio

In [8]:
doc = CadDocument('gear.jcad');

In [9]:
yobjects =doc.ydoc.get_array("objects")


In [4]:
async def update_color(target, color):
    doc.set_color(target, color)

In [5]:
NEW_COLOR_ACTION = []
def callback(target, events): 
    for event in events:
        if 'shapeMetadata' not in event.keys:
            target_name = event.target['name']
            asyncio.create_task( update_color(target_name, [1,0,0]))
            # NEW_COLOR_ACTION.append(dict(target=target_name, color=[1,0,0]))

In [7]:
# def after_transaction(e):
#     while NEW_COLOR_ACTION:
#         action = NEW_COLOR_ACTION.pop()
        

In [6]:
id = yobjects.observe_deep(partial(callback, "objects"))

In [9]:
# doc.ydoc.observe_after_transaction(after_transaction)

768736518

In [ ]:
yobjects.unobserve(id)

In [10]:
doc.set_color('Cylinder 2', None)